# IY011 Training Models Post Simulation

## Load the simulated results log and the trajectories

In [1]:
import os
import subprocess
import glob
import tempfile
from pathlib import Path
import numpy as np
import pandas as pd
import time
# plotting 
import matplotlib.pyplot as plt
from visualisation.plots import plot_mRNA_dist, plot_mRNA_trajectory
# simulation
from simulation.julia_simulate_telegraph_model import simulate_telegraph_model
# ml
import torch, itertools
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from classifiers.transformer_classifier import transformer_classifier
# data handling
from utils.data_loader import load_and_split_data
from utils.data_processing import add_binary_labels
from utils.standardise_time_series import standardise_time_series

%load_ext autoreload
%autoreload 2  

In [2]:
DATA_ROOT = Path("/home/ianyang/stochastic_simulations/experiments/EXP-25-IY011/data")
RESULTS_PATH = DATA_ROOT / "IY011_simulation_parameters_sobol.csv" #  this csv file stores all the simulation parameters used
df_params = pd.read_csv(RESULTS_PATH) 
# TRAJ_PATH = [DATA_ROOT / f"mRNA_trajectories_mu{row['mu_target']:.3f}_cv{row['cv_target']:.3f}_tac{row['t_ac_target']:.3f}.csv" for idx, row in df_params.iterrows()] # the trajectories 
TRAJ_PATH = [DATA_ROOT / df_params['trajectory_filename'].values[i] for i in range(len(df_params))]

# extract meta data
parameter_sets = [{
    'sigma_b': row['sigma_b'],
    'sigma_u': row['sigma_u'],
    'rho': row['rho'],
    'd': row['d'],
    'label': 0
} for idx, row in df_params.iterrows()]
time_points = np.arange(0, 3000, 1.0)
size = 1000

In [3]:
# read in the first trajectory file (csv)
df_traj = pd.read_csv(TRAJ_PATH[0])
df_traj = df_traj.drop(columns=['label'], errors='ignore') # drop in-place if it exists, do nothing otherwise
df_traj.head()

,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,...,t_2990,t_2991,t_2992,t_2993,t_2994,t_2995,t_2996,t_2997,t_2998,t_2999
0,0,13004,17205,17025,16838,16669,16494,16309,16163,15994,...,4270,4234,4199,4150,4108,4057,4014,3979,3944,3894
1,0,9950,9848,9730,9653,9562,9458,9383,9302,9211,...,1637,1617,1598,1584,1565,1554,1540,1522,1509,1499
2,0,13065,25840,26840,26586,26310,26008,25742,25479,25251,...,235,232,228,226,222,222,218,217,217,216
3,0,9142,9057,8965,8869,8785,8686,8583,8471,8395,...,63,63,63,63,62,60,57,56,55,55
4,0,3420,3383,3340,3302,3269,3245,3214,3185,3156,...,5,5,5,5,5,5,5,5,5,5


## label the datasets

In [4]:
label_column = 'mu_target'  # column name to base the binary labels on, mu_target for simplicity
labelled_df_params = add_binary_labels(df_params,label_column)
labels = [] 

for i in range(len(df_params)):
    # find the filename for each trajectory file
    trajectory_filename = df_params['trajectory_filename'].values[i]
    
    # get the corresponding label from labelled_df_params
    label_value = labelled_df_params[labelled_df_params['trajectory_filename'] == trajectory_filename]['label'].iloc[0]
    labels.append(label_value)

labels

[np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.int64(0),
 np.int64(0),
 np.int64(1),
 np.int64(1),
 np.in

Convert the datafiles from csv to NPZ format for faster loading in future <span style="color: red;">(this takes half an hour)</span>


In [ ]:
# convert the trajectory files (csv) into npz format
for traj_file, params, label in zip(TRAJ_PATH, parameter_sets, labels):
    df_traj = pd.read_csv(traj_file)
    df_traj = df_traj.drop(columns=['label'], errors='ignore') # drop in-place if it exists, do nothing otherwise
    trajectories = df_traj.values
    trajectory_data = {
            'trajectories': trajectories.astype(np.float32),
            'time_points': time_points.astype(np.float32),
            'size': int(size),
            'parameters': params,
            'label': label
    }
    try:
        np.savez_compressed(
        traj_file.with_suffix('.npz'),
        trajectories=trajectories.astype(np.float32),
        time_points=time_points.astype(np.float32),
        size=size,
        parameters=params,
        label=label
        )
    except PermissionError:
        with tempfile.NamedTemporaryFile(delete=False, suffix='.npz') as tmp_file:
            tmp_path = tmp_file.name
            np.savez_compressed(tmp_path, **trajectory_data)
        # Move temp file to final location with sudo
        subprocess.run(['sudo', 'mv', tmp_path, traj_file.with_suffix('.npz')], check=True)
        subprocess.run(['sudo', 'chown', f'{os.getenv("USER")}:{os.getenv("USER")}', traj_file.with_suffix('.npz')], check=True)


Each of the npz file will contain several arrays:
- trajectories
- time_points
- size
- parameters
- label (incorrectly named 'labels')

In [5]:
# load back the data (the first simulation) from the npz file
data = np.load(TRAJ_PATH[1].with_suffix('.npz'), allow_pickle=True)
print(data['trajectories'])
print(data['time_points'])
print(data['size'])
print(data['parameters'])
# print(data['labels']) # ....i made a typo here during the conversion

[[ 2053.  2023.  1994. ...  1433.  1411.  1386.]
 [ 8422.  8272.  8149. ... 23341. 22966. 22593.]
 [ 3147.  3098.  3047. ...   114.   114.   113.]
 ...
 [ 4381.  4308. 11701. ... 27395. 26960. 26544.]
 [14966. 14723. 14475. ... 44179. 43516. 42803.]
 [ 2659.  2624.  2583. ...  1603.  1570.  1541.]]
[1189. 1190. 1191. ... 2997. 2998. 2999.]
1000
{'sigma_b': 0.0118563038378397, 'sigma_u': 0.9881436961621602, 'rho': 11958.995981515953, 'd': 0.0163008923214477, 'label': 0}


In [6]:
# read traj back into dataframe
data_df = pd.DataFrame(data['trajectories'])
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810
0,2053.0,2023.0,1994.0,1966.0,1934.0,1901.0,1867.0,1835.0,1809.0,1773.0,...,1598.0,1562.0,1544.0,1526.0,1497.0,1476.0,1446.0,1433.0,1411.0,1386.0
1,8422.0,8272.0,8149.0,8021.0,7885.0,7756.0,7642.0,7519.0,7385.0,7268.0,...,26066.0,25679.0,25242.0,24844.0,24472.0,24069.0,23703.0,23341.0,22966.0,22593.0
2,3147.0,3098.0,3047.0,2996.0,2952.0,2890.0,2846.0,2798.0,2759.0,2717.0,...,125.0,122.0,120.0,119.0,117.0,115.0,114.0,114.0,114.0,113.0
3,34735.0,34131.0,33569.0,33025.0,32498.0,31955.0,31445.0,30949.0,30494.0,30011.0,...,7150.0,7037.0,6930.0,6805.0,6698.0,6591.0,6494.0,6374.0,6275.0,6160.0
4,11981.0,23682.0,25569.0,25158.0,24747.0,24352.0,23998.0,23623.0,23271.0,22902.0,...,468.0,462.0,456.0,452.0,445.0,442.0,433.0,427.0,420.0,412.0


concatenate all trajectories into one big npz file <span style="color: red;">(this takes about 20 minutes)</span>

In [ ]:
# Use the npz path
files = [str(p.with_suffix('.npz')) for p in TRAJ_PATH]

# Detect array keys from the first NPZ
# probe = np.load(files[0], allow_pickle=True)
# X_KEY = next(k for k in ["X", "trajectories", "M", "data"] if k in probe.files)
# Y_KEY = next(k for k in ["y", "labels", "label"] if k in probe.files)
# probe.close()
X_KEY = 'trajectories'
Y_KEY = 'labels'

# Load, stack, save
Xs, ys = [], []
for f in files:
    with np.load(f) as d:
        X, y = d[X_KEY], d[Y_KEY]
    if X.ndim == 2:            # add feature dim if missing
        X = X[..., None]       # -> (n, T, 1)
    n = X.shape[0]

    # --- make y shape (n,) regardless of how it’s stored ---
    y = np.asarray(y)
    if y.ndim == 0:                       # scalar label for the whole file
        y = np.full((n,), int(y), dtype=np.int64)
    else:
        y = y.reshape(-1)
        if y.shape[0] == 1 and n > 1:     # single value -> repeat for all n
            y = np.repeat(y.astype(np.int64), n)
        else:
            y = y.astype(np.int64)
    # --------------------------------------------------------

    Xs.append(X.astype(np.float32))
    ys.append(y)

X_all = np.concatenate(Xs, axis=0)   # (N_total, T, F)
y_all = np.concatenate(ys, axis=0)   # (N_total,)

In [ ]:
out_path = DATA_ROOT / "combined.npz"
try: 
    np.savez_compressed(out_path, X=X_all, y=y_all)
except PermissionError:
    with tempfile.NamedTemporaryFile(delete=False, suffix='.npz') as tmp_file:
        tmp_path = tmp_file.name
        np.savez_compressed(tmp_path, X=X_all, y=y_all)
    # Move temp file to final location with sudo
    subprocess.run(['sudo', 'mv', tmp_path, out_path], check=True)
    subprocess.run(['sudo', 'chown', f'{os.getenv("USER")}:{os.getenv("USER")}', out_path], check=True)

print("Saved:", out_path, "| Shapes:", X_all.shape, y_all.shape)


Saved: /home/ianyang/stochastic_simulations/experiments/EXP-25-IY011/data/combined.npz | Shapes: (1024000, 3000, 1) (1024000,)


## Load and split the combined data

In [7]:
COMBINED_NPZ_PATH = str(DATA_ROOT / "combined.npz")
# Use load_and_split_data for consistent splitting
X_train, X_val, X_test, y_train, y_val, y_test = load_and_split_data(
    COMBINED_NPZ_PATH, split_val_size=0.2
)

In [8]:
print(f"Data preparation:")
print(f"  Train shape: {X_train.shape}, Val shape: {X_val.shape}, Test shape: {X_test.shape}")
print(f"  Train labels: {y_train.shape}, Val labels: {y_val.shape}, Test labels: {y_test.shape}")

Data preparation:
  Train shape: (655360, 3000, 1), Val shape: (163840, 3000, 1), Test shape: (204800, 3000, 1)
  Train labels: (655360,), Val labels: (163840,), Test labels: (204800,)


## Model Training

since the data is quite large (1,024,000 samples of length 3000 each), we will initially only use a subset for training and testing
```
Memory Usage Comparison
Setup	Memory Usage	Training Speed
10k × 3000	~120 GB	3 min/epoch
1k × 3000	~12 GB	~18 sec/epoch
5k × 512	~10 GB	~15 sec/epoch
10k × 512	~20 GB	~30 sec/epoch
```

In [10]:
# Quick test with subset of data
subset_size = 1000  # Use only 1k samples

# Calculate subset sizes that preserve original ratios
train_ratio = len(X_train) / (len(X_train) + len(X_val) + len(X_test))
val_ratio = len(X_val) / (len(X_train) + len(X_val) + len(X_test))
test_ratio = len(X_test) / (len(X_train) + len(X_val) + len(X_test))

train_subset_size = int(subset_size * train_ratio)  # ~640
val_subset_size = int(subset_size * val_ratio)      # ~160
test_subset_size = int(subset_size * test_ratio)    # ~200

X_train_subset = X_train[:train_subset_size]
X_val_subset = X_val[:val_subset_size] 
X_test_subset = X_test[:test_subset_size]
y_train_subset = y_train[:train_subset_size]
y_val_subset = y_val[:val_subset_size]
y_test_subset = y_test[:test_subset_size]

print(f"New shapes: Train {X_train_subset.shape}, Val {X_val_subset.shape}, Test {X_test_subset.shape}")

transformer_accuracy = transformer_classifier(
    X_train_subset,
    X_val_subset, 
    X_test_subset,
    y_train_subset,
    y_val_subset,
    y_test_subset,
    d_model=64,  # Smaller model
    nhead=4,
    num_layers=2,  # Fewer layers
    batch_size=32,  
    epochs=10,  # Fewer epochs for testing
    use_conv1d=False,
    verbose=True
)

New shapes: Train (640, 3000, 1), Val (160, 3000, 1), Test (200, 3000, 1)
Starting training...
Epoch [1/10] | train_loss 0.61 | train_acc 0.80 | val_loss 0.51 | val_acc 0.81
Epoch [2/10] | train_loss 0.29 | train_acc 0.88 | val_loss 0.34 | val_acc 0.87
No improvement (1/10).
Epoch [3/10] | train_loss 0.25 | train_acc 0.90 | val_loss 0.30 | val_acc 0.87
No improvement (2/10).
Epoch [4/10] | train_loss 0.23 | train_acc 0.91 | val_loss 0.33 | val_acc 0.86
No improvement (3/10).
Epoch [5/10] | train_loss 0.22 | train_acc 0.92 | val_loss 0.35 | val_acc 0.87
Epoch [6/10] | train_loss 0.23 | train_acc 0.91 | val_loss 0.42 | val_acc 0.88
Epoch [7/10] | train_loss 0.22 | train_acc 0.91 | val_loss 0.53 | val_acc 0.88
No improvement (1/10).
Epoch [8/10] | train_loss 0.23 | train_acc 0.90 | val_loss 0.33 | val_acc 0.88
No improvement (2/10).
Epoch [9/10] | train_loss 0.24 | train_acc 0.91 | val_loss 0.33 | val_acc 0.88
No improvement (3/10).
Epoch [10/10] | train_loss 0.22 | train_acc 0.92 | val_l

TypeError: unsupported format string passed to tuple.__format__

The above approach still takes a long time, so let's try **truncating sequences**

Why Truncation Makes Sense:
1. Transformers struggle with very long sequences anyway (quadratic memory complexity)
2. Most temporal patterns in biological data are captured in shorter windows


In [ ]:
# Truncate sequences to manageable length
max_seq_length = 1024  # Much more reasonable for transformers

# Quick test with subset AND truncated sequences
subset_size = 10000  # Can use more samples now

train_subset_size = int(subset_size * train_ratio) 
val_subset_size = int(subset_size * val_ratio)      
test_subset_size = int(subset_size * test_ratio)  
  
X_train_subset = X_train[:train_subset_size, :max_seq_length, :]  # Truncate time dimension
X_val_subset = X_val[:val_subset_size, :max_seq_length, :]
X_test_subset = X_test[:test_subset_size, :max_seq_length, :]
y_train_subset = y_train[:train_subset_size]
y_val_subset = y_val[:val_subset_size]
y_test_subset = y_test[:test_subset_size]

print(f"New shapes: Train {X_train_subset.shape}, Val {X_val_subset.shape}, Test {X_test_subset.shape}")

transformer_accuracy = transformer_classifier(
    X_train_subset,
    X_val_subset, 
    X_test_subset,
    y_train_subset,
    y_val_subset,
    y_test_subset,
    d_model=128,
    nhead=4,
    num_layers=2,
    batch_size=64,  # Can use larger batches now
    epochs=20,
    use_conv1d=False,
    verbose=True
)

New shapes: Train (6400, 1024, 1), Val (1600, 1024, 1), Test (2000, 1024, 1)
🔄 Using device: cuda (1 GPUs available)
DEBUG: Optimizer initialized? True
✅ Running on CUDA!
Epoch [1/20], Loss: 1.4164, Train Acc: 0.6664
Validation Acc: 0.8144
Epoch [2/20], Loss: 0.5114, Train Acc: 0.8155
Validation Acc: 0.8256
Epoch [3/20], Loss: 0.4663, Train Acc: 0.8413
Validation Acc: 0.8369
Epoch [4/20], Loss: 0.4430, Train Acc: 0.8483
Validation Acc: 0.8419
Epoch [5/20], Loss: 0.4385, Train Acc: 0.8536
Validation Acc: 0.8438
Epoch [6/20], Loss: 0.4279, Train Acc: 0.8553
Validation Acc: 0.8419
No improvement (1/10).
Epoch [7/20], Loss: 0.4276, Train Acc: 0.8553
Validation Acc: 0.8369
No improvement (2/10).
Epoch [8/20], Loss: 0.4201, Train Acc: 0.8611
Validation Acc: 0.8419
No improvement (3/10).
Epoch [9/20], Loss: 0.4216, Train Acc: 0.8589
Validation Acc: 0.8456
Epoch [10/20], Loss: 0.4151, Train Acc: 0.8638
Validation Acc: 0.8406
No improvement (1/10).
Epoch [11/20], Loss: 0.4186, Train Acc: 0.8581

Truncation: More data but slightly lower performance - suggests truncation may have removed important patterns

So let's find out what the  optimal truncation length is

In [ ]:
# Test different sequence lengths
sequence_lengths = [256, 512, 1024, 1536, 2048]
results = {}

for seq_len in sequence_lengths:
    print(f"\n🔬 Testing sequence length: {seq_len}")
    
    X_train_seq = X_train[:5000, :seq_len, :]
    X_val_seq = X_val[:1250, :seq_len, :]
    X_test_seq = X_test[:1250, :seq_len, :]
    
    accuracy = transformer_classifier(
        X_train_seq, X_val_seq, X_test_seq,
        y_train[:5000], y_val[:1250], y_test[:1250],
        d_model=128, nhead=4, num_layers=2,
        batch_size=64, epochs=15, use_conv1d=False, verbose=False
    )
    
    results[seq_len] = accuracy
    print(f"✅ Sequence length {seq_len}: {accuracy:.3f}")

# Find optimal length
best_seq_len = max(results, key=results.get)
print(f"\n🎯 Best sequence length: {best_seq_len} (accuracy: {results[best_seq_len]:.3f})")


🔬 Testing sequence length: 256


=== Vanilla Transformer Accuracy: 0.80 ===
✅ Sequence length 256: 0.802

🔬 Testing sequence length: 512
=== Vanilla Transformer Accuracy: 0.83 ===
✅ Sequence length 512: 0.830

🔬 Testing sequence length: 1024
=== Vanilla Transformer Accuracy: 0.85 ===
✅ Sequence length 1024: 0.851

🔬 Testing sequence length: 1536
=== Vanilla Transformer Accuracy: 0.87 ===
✅ Sequence length 1536: 0.870

🔬 Testing sequence length: 2048
=== Vanilla Transformer Accuracy: 0.90 ===
✅ Sequence length 2048: 0.895

🎯 Best sequence length: 2048 (accuracy: 0.895)


In [ ]:
# Test different model sizes
configs = [
    {"d_model": 64, "nhead": 4, "num_layers": 2, "name": "Small"},
    {"d_model": 128, "nhead": 8, "num_layers": 2, "name": "Medium"},
    {"d_model": 256, "nhead": 8, "num_layers": 4, "name": "Large"},
    {"d_model": 128, "nhead": 4, "num_layers": 6, "name": "Deep"},
]

arch_results = {}
for config in configs:
    print(f"\n🏗️ Testing {config['name']} model")
    
    accuracy = transformer_classifier(
        X_train_subset, X_val_subset, X_test_subset,
        y_train_subset, y_val_subset, y_test_subset,
        d_model=config["d_model"],
        nhead=config["nhead"], 
        num_layers=config["num_layers"],
        batch_size=32, epochs=15, use_conv1d=False, verbose=False
    )
    
    arch_results[config["name"]] = accuracy
    print(f"✅ {config['name']}: {accuracy:.3f}")


🏗️ Testing Small model
=== Vanilla Transformer Accuracy: 0.85 ===
✅ Small: 0.854

🏗️ Testing Medium model
=== Vanilla Transformer Accuracy: 0.85 ===
✅ Medium: 0.855

🏗️ Testing Large model
=== Vanilla Transformer Accuracy: 0.86 ===
✅ Large: 0.858

🏗️ Testing Deep model
=== Vanilla Transformer Accuracy: 0.86 ===
✅ Deep: 0.857
